# Using web scrapping skills to get recent news about the 2024 Taiwan President Election

In [ ]:
# Set up for running selenium in Google Colab
## You don't need to run this code if you do it in Jupyter notebook, or other local Python setting
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,344 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [662 kB]
Get:13 http://security.ubuntu

In [ ]:
!pip install chromedriver-autoinstaller

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

# set the target URL
url = "put-url-here-to-scrape"

# set up the webdriver
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime, timedelta
import re

In [ ]:
def get_main_element (main_element_id, driver):
  main_elements = driver.find_elements(By.CSS_SELECTOR, main_element_id)
  return main_elements

def get_row_data(main_element):

  # Extract text content from nested elements
  try:
    internal_link_text = main_element.find_element(By.CSS_SELECTOR, "a[data-testid='internal-link']").get_attribute("href")
    headline_text = main_element.find_element(By.CSS_SELECTOR, "h2[data-testid='card-headline']").text
    description_text = main_element.find_element(By.CSS_SELECTOR, "p[data-testid='card-description']").text
    last_updated_element = main_element.find_element(By.CSS_SELECTOR, "span[data-testid='card-metadata-lastupdated']").text
    return [headline_text,description_text,last_updated_element,internal_link_text]
  except:
    return []

In [ ]:
driver = webdriver.Chrome(options=chrome_options)

search_query = "taiwan election"
driver.get(f"https://www.bbc.com/search?q={search_query}")
next_page_element="button[data-testid='pagination-next-button']"
headline_card_element= "h2[data-testid='card-headline']"
page= 1
main_element_id="div[data-testid='liverpool-card']"
total_list=[]

while EC.presence_of_element_located((By.CSS_SELECTOR, main_element_id)):

  print(f'Page: {page}')

  # Find all elements with the specified HTML tag and attributes
  temp_main_elements=get_main_element(main_element_id, driver)

  for main_element in temp_main_elements:

    temp_row= get_row_data(main_element)
    total_list.append(temp_row)

  next_button = driver.find_element(By.CSS_SELECTOR, next_page_element)
  next_button.click()


  try:
      element = WebDriverWait(driver, 10).until(
          EC.presence_of_element_located((By.CSS_SELECTOR, main_element_id))
      )
      # Perform actions on the element once it's present
      #print("Element found:", element.text)
      page+=1
  except TimeoutException:
      print("Timed out waiting for the element")

  if page>10:
    break



# Close the browser window
driver.quit()

Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10


In [ ]:
#sanity check
for item in total_list:
  print(item)

print(len(total_list))

['DPP supporters celebrate victory in Taiwan election', 'Democratic Progressive Party supporters celebrate at a rally as their opponents concede in the Taiwan election.', '4 days ago', 'https://www.bbc.com/news/world-asia-67967549']
['Why the Taiwan election matters to the world', "It will redefine Taiwan's relationship with China and have implications for key allies like the US.", '4 days ago', 'https://www.bbc.com/news/world-asia-67871587']
[]
['Taiwan elects pro-sovereignty leader, angering Beijing', 'The move has angered Beijing which reviles Mr Lai and his pro-sovereignty party.', '4 days ago', 'https://www.bbc.com/news/world-asia-67920532']
["China says US 'gravely wrong' to congratulate Taiwan leader", 'Beijing accuses the US of violating its commitment to maintain only unofficial ties with Taiwan.', '3 days ago', 'https://www.bbc.com/news/world-asia-67974541']
["China warns Taiwan voters and condemns 'brazen' US", 'China says the ruling party candidate is a risk to relations, t

In [ ]:
#Convert raw data in to dataframe

df_raw = pd.DataFrame(total_list, columns =["Headline","Description","Date","href"])
df_raw['index_column'] = df_raw.index
index_column = df_raw.pop('index_column')
df_raw.insert(0, 'id', index_column)
print(df_raw.head(10))

   id                                           Headline  \
0   0  DPP supporters celebrate victory in Taiwan ele...   
1   1       Why the Taiwan election matters to the world   
2   2                                               None   
3   3  Taiwan elects pro-sovereignty leader, angering...   
4   4  China says US 'gravely wrong' to congratulate ...   
5   5  China warns Taiwan voters and condemns 'brazen...   
6   6  China tells US it will 'never compromise' on T...   
7   7  Taiwan just chose a president China loathes. W...   
8   8  The Taiwan party asking voters to choose war o...   
9   9  It's not war Taiwan's young voters worry about...   

                                         Description         Date  \
0  Democratic Progressive Party supporters celebr...   4 days ago   
1  It will redefine Taiwan's relationship with Ch...   4 days ago   
2                                               None         None   
3  The move has angered Beijing which reviles Mr ...   4 days a

In [ ]:
#check for Null

print (df_raw.isnull().sum())
df_copy= df_raw.copy()

#remove Null value
df_copy=df_copy.dropna()


id             0
Headline       1
Description    1
Date           1
href           1
dtype: int64


In [ ]:
def adjust_Date(date):

  pattern = r'(\d+)\s*days?\s+ago'

    # Use re.search to find the pattern in the text
  match = re.search(pattern, date)

  if re.search(pattern, date):
    current_date = datetime.now()
    result_date = current_date - timedelta(days=int(match.group(1)))
    return result_date.strftime("%d %b %Y")
  else:
    return date

#sanity check
print(adjust_Date('4 days ago'))
print(adjust_Date('13 Jan 2024'))

13 Jan 2024
13 Jan 2024


In [ ]:
#adjust date column in df
df_copy['Date']=df_copy['Date'].apply(adjust_Date)

#sanity check
print(df_copy)

    id                                           Headline  \
0    0  DPP supporters celebrate victory in Taiwan ele...   
1    1       Why the Taiwan election matters to the world   
3    3  Taiwan elects pro-sovereignty leader, angering...   
4    4  China says US 'gravely wrong' to congratulate ...   
5    5  China warns Taiwan voters and condemns 'brazen...   
..  ..                                                ...   
85  85    Angry exchanges at US and China talks in Alaska   
86  86                                    Vanuatu profile   
87  87    Trump moves to pull US out of WHO amid pandemic   
88  88    How India calculates value of women's housework   
89  89          NZ reacts to 'creepy' PM Ardern interview   

                                          Description         Date  \
0   Democratic Progressive Party supporters celebr...  13 Jan 2024   
1   It will redefine Taiwan's relationship with Ch...  13 Jan 2024   
3   The move has angered Beijing which reviles Mr ...  13

In [ ]:
#conver to datetime object for date column
df_copy['Date'] = pd.to_datetime(df_copy['Date'])
print(df_copy)

    id                                           Headline  \
0    0  DPP supporters celebrate victory in Taiwan ele...   
1    1       Why the Taiwan election matters to the world   
3    3  Taiwan elects pro-sovereignty leader, angering...   
4    4  China says US 'gravely wrong' to congratulate ...   
5    5  China warns Taiwan voters and condemns 'brazen...   
..  ..                                                ...   
85  85    Angry exchanges at US and China talks in Alaska   
86  86                                    Vanuatu profile   
87  87    Trump moves to pull US out of WHO amid pandemic   
88  88    How India calculates value of women's housework   
89  89          NZ reacts to 'creepy' PM Ardern interview   

                                          Description       Date  \
0   Democratic Progressive Party supporters celebr... 2024-01-13   
1   It will redefine Taiwan's relationship with Ch... 2024-01-13   
3   The move has angered Beijing which reviles Mr ... 2024-01-1

In [ ]:
# filter entries for the past 2 months

start_date_str = '2023-11-01'
end_date_str = datetime.now()

# Convert the strings to datetime objects
start_date = pd.to_datetime(start_date_str)
end_date = pd.to_datetime(end_date_str)

# filter by date
filtered_df_date = df_copy.loc[(df_copy['Date'] >= start_date) & (df_copy['Date'] <= end_date)]


In [ ]:
#sanity check
print(filtered_df_date)

    id                                           Headline  \
0    0  DPP supporters celebrate victory in Taiwan ele...   
1    1       Why the Taiwan election matters to the world   
3    3  Taiwan elects pro-sovereignty leader, angering...   
4    4  China says US 'gravely wrong' to congratulate ...   
5    5  China warns Taiwan voters and condemns 'brazen...   
6    6  China tells US it will 'never compromise' on T...   
7    7  Taiwan just chose a president China loathes. W...   
8    8  The Taiwan party asking voters to choose war o...   
9    9  It's not war Taiwan's young voters worry about...   
10  10  Chinese satellite triggers missile alert in Ta...   
11  11  The dark horse who has shaken up Taiwan's poli...   
12  12     Nauru cuts ties with Taiwan in favour of China   
13  13           The Taiwan that China wants is vanishing   
14  14  The dark rumours trying to destroy Taiwan's tr...   
15  15  Love and longing across the Taiwan Strait: The...   
16  16  How popcorn chic

In [ ]:
#filter by title actaully related to Taiwan
filtered_df_name = filtered_df_date[filtered_df_date['Headline'].str.contains('taiwan', case=False)]

#sanity check
print(filtered_df_name)

    id                                           Headline  \
0    0  DPP supporters celebrate victory in Taiwan ele...   
1    1       Why the Taiwan election matters to the world   
3    3  Taiwan elects pro-sovereignty leader, angering...   
4    4  China says US 'gravely wrong' to congratulate ...   
5    5  China warns Taiwan voters and condemns 'brazen...   
6    6  China tells US it will 'never compromise' on T...   
7    7  Taiwan just chose a president China loathes. W...   
8    8  The Taiwan party asking voters to choose war o...   
9    9  It's not war Taiwan's young voters worry about...   
10  10  Chinese satellite triggers missile alert in Ta...   
11  11  The dark horse who has shaken up Taiwan's poli...   
12  12     Nauru cuts ties with Taiwan in favour of China   
13  13           The Taiwan that China wants is vanishing   
14  14  The dark rumours trying to destroy Taiwan's tr...   
15  15  Love and longing across the Taiwan Strait: The...   
16  16  How popcorn chic

In [ ]:
# save as csv
filtered_df_name.to_csv('taiwan_2024_president_election_news.csv',index=False)